## Statsmodel Forecast AB Testing

A/B Testing is one method of models against each other.  This demonstration will show how to use the Wallaroo pipeline step `add_random_split` and `replace_with_random_split` to randomly submit inference input data into control and challenger models.

## Prerequisites

* A Wallaroo instance version 2023.2.1 or greater.

## References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Python Models](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)
* [Wallaroo SDK Essentials: Inference Guide: Parallel Inferences](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-inferences/#parallel-inferences)

## A/B Testing

A/B testing is a method that provides the ability to test out ML models for performance, accuracy or other useful benchmarks.  A/B testing is contrasted with the Wallaroo Shadow Deployment feature.  In both cases, two sets of models are added to a pipeline step:

* Control or Champion model:  The model currently used for inferences.
* Challenger model(s): One or more models that are to be compared to the champion model.

The two feature are different in this way:

| Feature | Description |
|---|---|
| A/B Testing | A subset of inferences are submitted to either the champion ML model or a challenger ML model. |
| Shadow Deploy | All inferences are submitted to the champion model and one or more challenger models. |

Wallaroo implements A/B testing via a pipeline step as either `wallaroo.pipeline.add_random_split` to set a new pipeline step or `wallaroo.pipeline.replace_with_random_split` to replace an existing step.

## Tutorial Steps

### Import Libraries

The first step is to import the libraries that we will need.

In [73]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
import numpy as np

from resources import simdb
from resources import util

pd.set_option('display.max_colwidth', None)

In [74]:
display(wallaroo.__version__)

'2023.2.1'

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [75]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

Workspace names must be unique.  To allow this tutorial to run in the same Wallaroo instance for multiple users, set the `suffix` variable or share the workspace with other users.

#### Set Configurations References

* [Wallaroo SDK Essentials Guide: Workspace Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-workspace/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)

In [76]:
# used for unique connection names

suffix='jch'

workspace_name = f'forecast-model-workshop{suffix}'

pipeline_name = 'forecast-workshop-pipeline'

### Set the Workspace, Pipeline and Models

The workspace will be either used or created if it does not exist, along with the pipeline.

The models were uploaded in the Upload and Deploy notebook.

In [77]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

# Get the most recent version of a model in the workspace
# Assumes that the most recent version is the first in the list of versions.
# wl.get_current_workspace().models() returns a list of models in the current workspace

def get_model(mname):
    modellist = wl.get_current_workspace().models()
    model = [m.versions()[0] for m in modellist if m.name() == mname]
    if len(model) <= 0:
        raise KeyError(f"model {mname} not found in this workspace")
    return model[0]

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

# upload three models:  the control and two challengers

control_model_name = 'forecast-control-model'
challenger01_model_name = 'forecast-challenger01-model'
challenger02_model_name = 'forecast-challenger02-model'

# upload the models

bike_day_model = get_model(control_model_name)

challenger_model_01 = get_model(challenger01_model_name)

challenger_model_02 = get_model(challenger02_model_name)


### Deploy the Pipeline

We will now add the uploaded model as a step for the pipeline, then deploy it.  The pipeline configuration will allow for multiple replicas of the pipeline to be deployed and spooled up in the cluster.  Each pipeline replica will use 0.25 cpu and 512 Gi RAM.

In [78]:
# Set the deployment to allow for additional engines to run
# Undeploy and clear the pipeline in case it was used in other demonstrations
pipeline.undeploy()
pipeline.clear()
deploy_config = (wallaroo.DeploymentConfigBuilder()
                        .replica_count(1)
                        .replica_autoscale_min_max(minimum=2, maximum=5)
                        .cpus(0.25)
                        .memory("512Mi")
                        .build()
                    )

pipeline.add_model_step(bike_day_model)
# pipeline.add_model_step(step)
pipeline.deploy(deployment_config = deploy_config)

name,forecast-workshop-pipeline
created,2023-07-27 15:54:55.416132+00:00
last_updated,2023-07-27 16:08:39.633323+00:00
deployed,True
tags,
versions,"dc172ae6-00f1-4cdc-8b19-591eaeb72185, 02039466-5f92-40c0-a846-2944cb0cb995, 25d73dc4-00c0-4d09-b897-f8cb7df45ec0, e3a29c1e-4922-4bd5-8fe8-55b8eed2df31, ace2bfea-1c99-45bb-bd46-236600f78e11, 404e87ac-94e2-43f8-bd75-9c7ff76ad7d2, 41298f7f-d353-49f6-ad34-ef251ea321cf"
steps,forecast-challenger02-model


### Run Inference

Run a test inference to verify the pipeline is operational from the sample test data stored in `./data/testdata_dict.json`.

In [79]:
inferencedata = json.load(open("./data/testdata_dict.json"))

results = pipeline.infer(inferencedata)

display(results)

[{'forecast': [1764, 1749, 1743, 1741, 1740, 1740, 1740]}]

### Create A/B Step

Here we will configure a pipeline with two models and set the control model with a random split chance of receiving 2/3 of the data.  Because this is a random split, it is possible for one model or the other to receive more inferences than a strict 2:1 ratio, but the more inferences are run, the more likely it is for the proper ratio split.

The control model was already set as a python step, so we will be replacing it with the random split step, then deploying the pipeline to set the configuration.

In [80]:
pipeline.replace_with_random_split(0, 
                                   [(2, bike_day_model), 
                                    (1, challenger_model_01), 
                                    (1, challenger_model_02)], 
                                    "session_id"
                                    )
pipeline.deploy()

name,forecast-workshop-pipeline
created,2023-07-27 15:54:55.416132+00:00
last_updated,2023-07-27 16:09:26.962727+00:00
deployed,True
tags,
versions,"351736d9-07bf-4956-9724-6e135be5fcd8, dc172ae6-00f1-4cdc-8b19-591eaeb72185, 02039466-5f92-40c0-a846-2944cb0cb995, 25d73dc4-00c0-4d09-b897-f8cb7df45ec0, e3a29c1e-4922-4bd5-8fe8-55b8eed2df31, ace2bfea-1c99-45bb-bd46-236600f78e11, 404e87ac-94e2-43f8-bd75-9c7ff76ad7d2, 41298f7f-d353-49f6-ad34-ef251ea321cf"
steps,forecast-challenger02-model


### Inference via Random Split

A sample inference will be run.  Because this is a model that accepts JSON as the input and outputs, we will not see which model was used in the inference results.

For the later log steps, we will run the inference 10 times to view the results, and how they change with the same input depending on which model the inference request is submitted to.

In [81]:
inferencedata = json.load(open("./data/testdata_dict.json"))

for i in range(10):
    results = pipeline.infer(inferencedata)
    display(results)

[{'forecast': [1814, 1814, 1814, 1814, 1814, 1814, 1814]}]

[{'forecast': [1814, 1814, 1814, 1814, 1814, 1814, 1814]}]

[{'forecast': [1814, 1814, 1814, 1814, 1814, 1814, 1814]}]

[{'forecast': [1764, 1749, 1743, 1741, 1740, 1740, 1740]}]

[{'forecast': [1814, 1814, 1814, 1814, 1814, 1814, 1814]}]

[{'forecast': [1703, 1757, 1737, 1744, 1742, 1743, 1742]}]

[{'forecast': [1814, 1814, 1814, 1814, 1814, 1814, 1814]}]

[{'forecast': [1764, 1749, 1743, 1741, 1740, 1740, 1740]}]

[{'forecast': [1703, 1757, 1737, 1744, 1742, 1743, 1742]}]

[{'forecast': [1703, 1757, 1737, 1744, 1742, 1743, 1742]}]

To view what model was used for the inference, we look at the pipeline logs with the dataset `metadata` to retrieve additional information.

The following will retrieves the `model_name` parameter from the `metadata.last_model` metadata.  This will allow us to easily identify which model was used with which inference request and output.

In [82]:
def get_log_model(df: pd.DataFrame):
    return df['metadata.last_model'].apply(lambda x: json.loads(x)['model_name'])

In [83]:
logs = pipeline.logs(limit=10, dataset=["time", "out.json","metadata"])
logs['model'] = get_log_model(logs)

logs.loc[:, ["time", "out.json", "model"]]

,time,out.json,model
0,2023-07-27 16:10:05.054,"{""forecast"":[1703,1757,1737,1744,1742,1743,1742]}",forecast-challenger01-model
1,2023-07-27 16:10:04.648,"{""forecast"":[1764,1749,1743,1741,1740,1740,1740]}",forecast-control-model
2,2023-07-27 16:10:03.806,"{""forecast"":[1703,1757,1737,1744,1742,1743,1742]}",forecast-challenger01-model
3,2023-07-27 16:10:02.480,"{""forecast"":[1814,1814,1814,1814,1814,1814,1814]}",forecast-challenger02-model
4,2023-07-27 16:10:02.056,"{""forecast"":[1814,1814,1814,1814,1814,1814,1814]}",forecast-challenger02-model
5,2023-07-27 16:10:01.618,"{""forecast"":[1814,1814,1814,1814,1814,1814,1814]}",forecast-challenger02-model
6,2023-07-27 16:07:07.019,"{""forecast"":[1814,1814,1814,1814,1814,1814,1814]}",forecast-challenger02-model
7,2023-07-27 16:07:06.582,"{""forecast"":[1764,1749,1743,1741,1740,1740,1740]}",forecast-control-model
8,2023-07-27 16:07:06.149,"{""forecast"":[1764,1749,1743,1741,1740,1740,1740]}",forecast-control-model
9,2023-07-27 16:07:05.268,"{""forecast"":[1764,1749,1743,1741,1740,1740,1740]}",forecast-control-model


### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [84]:
pipeline.undeploy()

name,forecast-workshop-pipeline
created,2023-07-27 15:54:55.416132+00:00
last_updated,2023-07-27 16:09:26.962727+00:00
deployed,False
tags,
versions,"351736d9-07bf-4956-9724-6e135be5fcd8, dc172ae6-00f1-4cdc-8b19-591eaeb72185, 02039466-5f92-40c0-a846-2944cb0cb995, 25d73dc4-00c0-4d09-b897-f8cb7df45ec0, e3a29c1e-4922-4bd5-8fe8-55b8eed2df31, ace2bfea-1c99-45bb-bd46-236600f78e11, 404e87ac-94e2-43f8-bd75-9c7ff76ad7d2, 41298f7f-d353-49f6-ad34-ef251ea321cf"
steps,forecast-challenger02-model
